                                                               Assignment-2

Name: Arman Garg

Roll_no: 102356002

Grp: P1B

In [ ]:
!nvidia-smi

Mon Feb  2 10:50:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


**Q1**
Identify !, %, and %% used in cell in Google Colab.

In [2]:
!ls
%time a = 10 + 20

sample_data
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


In [3]:
%%writefile hello.cu
#include <stdio.h>
__global__ void hello() {
    printf("Hello from GPU!\n");
}

int main() {
    hello<<<1, 1>>>();
    cudaDeviceSynchronize();
    printf("Hello from CPU!\n");
    return 0;
}

Writing hello.cu


In [5]:
!nvcc -arch=sm_75 hello.cu -o hello

In [6]:
!./hello

Hello from GPU!
Hello from CPU!


**Q2**
Identify all key nvidia-smi commands with multiple options

In [9]:
!nvidia-smi

!nvidia-smi -L

!nvidia-smi -q

Sat Feb 21 17:44:42 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             14W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Q3**
Debug common CUDA errors (zero output, incorrect indexing, PTX errors)

In [31]:
%%writefile q3.cu
#include <stdio.h>

__global__ void kernel(int *d) {
    int id = threadIdx.x;              // Error: incorrect indexing
    d[id] = id;
}

int main() {
    int h[2];
    int *d;
    cudaMalloc(&d, 2 * sizeof(int));
    kernel<<<1,2>>>(d);                // Error: kernel launch is asynchronous
    cudaMemcpy(h, d, 2 * sizeof(int), cudaMemcpyDeviceToHost); // Error: no cudaDeviceSynchronize before copy
    printf("%d\n", h[0]);     // Error:zero value
    return 0;
}

Overwriting q3.cu


In [32]:
!nvcc -arch=sm_75 q3.cu -o q3
!./q3

0


In [36]:
%%writefile q31.cu
#include <stdio.h>

__global__ void kernel(int *d) {
    int id = blockIdx.x * blockDim.x + threadIdx.x;
    d[id] = id;
}

int main() {
    int h[2];
    int *d;
    cudaMalloc(&d, 2 * sizeof(int));
    kernel<<<1,2>>>(d);
    cudaDeviceSynchronize();
    cudaMemcpy(h, d, 2 * sizeof(int), cudaMemcpyDeviceToHost);
    printf("%d %d\n", h[0], h[1]);
    cudaFree(d);
    return 0;
}

Writing q31.cu


In [37]:
!nvcc -arch=sm_75 q31.cu -o q31
!./q31

0 1


**Q4**

Write a CUDA C/C++ program to demonstrate GPU kernel execu'on and thread indexing.
a. Launch a CUDA kernel using: 1 block and 8 threads

b. Each thread must print: Hello from GPU thread <global_thread_id>

c. Compute the global thread ID using: global_thread_id = blockIdx.x * blockDim.x +
threadIdx.x

d. Clearly separate: Host code (CPU) & Device code (GPU kernel)

In [14]:
%%writefile q4.cu
#include <stdio.h>

__global__ void gpuKernel() {
    int global_thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Hello from GPU thread %d\n", global_thread_id);
}

int main() {
    gpuKernel<<<1,8>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing q4.cu


In [15]:
!nvcc -arch=sm_75 q4.cu -o q4

In [16]:
!./q4

Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from GPU thread 4
Hello from GPU thread 5
Hello from GPU thread 6
Hello from GPU thread 7


**Q5**

Write a CUDA program to demonstrate host and device memory separa'on.
a. Create an integer array of size 5 on the host (CPU).

b. Allocate corresponding memory on the device (GPU) using cudaMalloc().

c. Copy data from host to device using cudaMemcpy().

d. Launch a kernel where GPU threads print values from device memory.

e. Copy the data back from device to host and print it on CPU.

In [17]:
%%writefile q5.cu
#include <stdio.h>

__global__ void showData(int *d) {
    int id = threadIdx.x;
    printf("%d\n", d[id]);
}

int main() {
    int h[5] = {10,20,30,40,50};
    int *d;
    cudaMalloc(&d, 5 * sizeof(int));
    cudaMemcpy(d, h, 5 * sizeof(int), cudaMemcpyHostToDevice);
    showData<<<1,5>>>(d);
    cudaMemcpy(h, d, 5 * sizeof(int), cudaMemcpyDeviceToHost);
    for(int i=0;i<5;i++) printf("%d\n", h[i]);
    cudaFree(d);
    return 0;
}

Writing q5.cu


In [18]:
!nvcc -arch=sm_75 q5.cu -o q5
!./q5

10
20
30
40
50
10
20
30
40
50


**Q6**
Compare CPU times of List/tuple with Numpy arrays.

In [20]:
import numpy as np
import time

lst = list(range(1000000))
arr = np.array(lst)

t1 = time.time()
s = [x*2 for x in lst]
print("time for list/tuple:", time.time() - t1)

t2 = time.time()
s = arr * 2
print("time for cpu:", time.time() - t2)

time for list/tuple: 0.04410099983215332
time for cpu: 0.01569962501525879
